In [1]:
!pip install chromadb sentence-transformers transformers torch pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.4/132.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 5.4 MB/s eta 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import chromadb
from chromadb.utils import embedding_functions
import json
import pandas as pd
from transformers import pipeline
import torch

In [4]:
# ===== SETUP =====
print("Setting up ChromaDB...")
client = chromadb.Client()
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

collection = client.create_collection(
    name="cooking_docs",
    embedding_function=sentence_transformer_ef
)

Setting up ChromaDB...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
# Load LLM (only once at startup)
print("Loading language model (this may take a minute)...")
device = 0 if torch.cuda.is_available() else -1  # Use GPU if available
generator = pipeline(
    'text-generation',
    model='TinyLlama/TinyLlama-1.1B-Chat-v1.0',
    device=device
)
print(f"Model loaded on: {'GPU' if device == 0 else 'CPU'}")

Loading language model (this may take a minute)...


Device set to use cpu


Model loaded on: CPU


In [36]:
# ===== CHUNKING FUNCTIONS =====

def chunk_fcs_json(data, source_file):
    chunks = []

    # Introduction
    chunks.append({
        'text': f"{data['title']}\n\n{data['introduction']}",
        'metadata': {'type': 'introduction', 'source': source_file}
    })

    # Cooking methods and techniques
    for method_name, method_data in data['cooking_methods'].items():
        if isinstance(method_data, dict) and 'techniques' in method_data:
            for technique_name, technique_desc in method_data['techniques'].items():
                chunks.append({
                    'text': f"Cooking Method: {method_name.replace('_', ' ').title()}\n\n"
                            f"{method_data['description']}\n\n"
                            f"Technique: {technique_name}\n{technique_desc}",
                    'metadata': {
                        'type': 'technique',
                        'method': method_name,
                        'technique': technique_name,
                        'source': source_file
                    }
                })
        else:
            chunks.append({
                'text': f"Cooking Method: {method_name.title()}\n\n{method_data['description']}",
                'metadata': {'type': 'method', 'method': method_name, 'source': source_file}
            })

    # Kitchen tools
    if 'kitchen_tools' in data:
        for category, category_data in data['kitchen_tools'].items():
            for tool_name, tool_desc in category_data['items'].items():
                chunks.append({
                    'text': f"Kitchen Tool: {tool_name}\n\n{tool_desc}",
                    'metadata': {
                        'type': 'kitchen_tool',
                        'category': category,
                        'tool': tool_name,
                        'source': source_file
                    }
                })

    return chunks


def chunk_healthy_cooking(data, source_file):
    chunks = []

    for method_name, method_desc in data['cooking_methods'].items():
        chunks.append({
            'text': f"Healthy Cooking Method: {method_name}\n\n{method_desc}",
            'metadata': {
                'type': 'healthy_method',
                'method': method_name.lower(),
                'source': source_file
            }
        })

    if 'food_preparation_tips' in data:
        tips_text = "Healthy Food Preparation Tips:\n\n" + "\n\n".join(
            f"• {tip}" for tip in data['food_preparation_tips']
        )
        chunks.append({
            'text': tips_text,
            'metadata': {'type': 'preparation_tips', 'source': source_file}
        })

    return chunks


def chunk_recipe_json(data, source_file):
    ingredients_text = "\n".join(
        f"- {ing['item']}: {ing['quantity']}"
        for ing in data['ingredients']
    )

    recipe_text = f"""Recipe: {data['dish_name']}

Prep time: {data.get('prep_time', 'N/A')}
Cooking time: {data.get('cooking_time', 'N/A')}
Portions: {data.get('portions', 'N/A')}

Ingredients:
{ingredients_text}"""

    return [{
        'text': recipe_text,
        'metadata': {
            'type': 'recipe',
            'dish_name': data['dish_name'],
            'source': source_file
        }
    }]


def chunk_csv_simple(file_path):
    """
    Put entire CSV in one chunk (for small CSVs ~5 lines)
    """
    try:
        df = pd.read_csv(file_path)

        # Format as readable text
        text_parts = [f"Data from {file_path.split('/')[-1]}:\n"]

        for idx, row in df.iterrows():
            row_text = " | ".join([f"{col}: {val}" for col, val in row.items() if pd.notna(val)])
            text_parts.append(row_text)

        text = "\n".join(text_parts)

        return [{
            'text': text,
            'metadata': {
                'type': 'csv_data',
                'source': file_path,
                'filename': file_path.split('/')[-1],
                'rows': len(df),
                'columns': len(df.columns)
            }
        }]

    except Exception as e:
        print(f"  ✗ Error processing CSV {file_path}: {e}")
        return []

def chunk_json_simple(file_path):
    """
    Put each row of JSON in its own chunk - subject: text related to subject format
    """

    chunks = []

    try:
        with open(file_path, 'r') as f:
            data = json.load(f)

        for key, val in data.items():
                chunks.append({
                    'text': f"{key}: {val}",
                    'metadata': {
                        'type': 'json_data',
                        'source': file_path,
                        'key': key,
                        'value': val
                    }
                })

    except Exception as e:
        print(f"  ✗ Error processing JSON {file_path}: {e}")
        return []

    return chunks

In [37]:
# ===== INGESTION =====

def ingest_all_documents():
    all_chunks = []

    # Load core technique files
    print("Loading core cooking technique files...")
    specific_files = {
        '/content/drive/MyDrive/final-project/FCS.json': chunk_fcs_json,
        '/content/drive/MyDrive/final-project/healthy_cooking_method.json': chunk_healthy_cooking,
    }

    for file_path, chunk_function in specific_files.items():
        try:
            with open(file_path, 'r') as f:
                data = json.load(f)
            chunks = chunk_function(data, file_path)
            all_chunks.extend(chunks)
            print(f"✓ Loaded {len(chunks)} chunks from {file_path.split('/')[-1]}")
        except Exception as e:
            print(f"✗ Error loading {file_path}: {e}")

    # Load all recipes from foc folder
    print("\nLoading recipes from foc folder...")
    foc_folder = '/content/drive/MyDrive/final-project/foc'
    recipe_count = 0

    if os.path.exists(foc_folder):
        foc_json_files = glob.glob(os.path.join(foc_folder, '*.json'))

        for file_path in foc_json_files:
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                try:
                    chunks = chunk_recipe_json(data, file_path)
                    all_chunks.extend(chunks)
                    recipe_count += 1
                except:
                    chunks = chunk_json_simple(file_path)
                    all_chunks.extend(chunks)

            except Exception as e:
                print(f"  ✗ Error loading {file_path.split('/')[-1]}: {e}")

        print(f"✓ Loaded {recipe_count} recipes")
    else:
        print(f"✗ foc folder not found")

    # Load all CSV files (one chunk per file)
    print("\nLoading CSV files...")
    project_folder = '/content/drive/MyDrive/final-project'
    csv_files = glob.glob(os.path.join(project_folder, '*.csv'))
    csv_count = 0

    for csv_file in csv_files:
        csv_chunks = chunk_csv_simple(csv_file)
        if csv_chunks:
            all_chunks.extend(csv_chunks)
            csv_count += 1
            print(f"✓ Loaded {csv_file.split('/')[-1]}")

    # Add all chunks to ChromaDB
    if all_chunks:
        documents = [chunk['text'] for chunk in all_chunks]
        metadatas = [chunk['metadata'] for chunk in all_chunks]
        ids = [f"doc_{i}" for i in range(len(all_chunks))]

        collection.add(
            documents=documents,
            metadatas=metadatas,
            ids=ids
        )

        print(f"\n{'='*60}")
        print(f"✓ INGESTION COMPLETE")
        print(f"{'='*60}")
        print(f"Total chunks: {len(all_chunks)}")
        print(f"  - Recipes: {recipe_count}")
        print(f"  - CSV files: {csv_count}")
        print(f"  - Other: {len(all_chunks) - recipe_count - csv_count}")
        print(f"{'='*60}")
    else:
        print("✗ No chunks to ingest!")

    return len(all_chunks)

In [38]:
# ===== QUERY & GENERATE =====

def query_rag(question, n_results=3):
    """Retrieve relevant chunks"""
    results = collection.query(
        query_texts=[question],
        n_results=n_results
    )
    return results['documents'][0], results['metadatas'][0]


def ask_cooking_assistant(question):
    """Main RAG function with Hugging Face"""
    # Retrieve relevant docs
    contexts, metadatas = query_rag(question, n_results=3)
    context_text = "\n\n---\n\n".join(contexts)

    # Build prompt
    prompt = f"""<|system|>
You are a helpful cooking assistant. Answer questions based on the provided context. Be concise and accurate.<|end|>
<|user|>
Context:
{context_text}

Question: {question}<|end|>
<|assistant|>"""

    # Generate response
    result = generator(
        prompt,
        max_new_tokens=300,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1,
        return_full_text=False
    )

    answer = result[0]['generated_text'].strip()
    sources = list(set([m['source'].split('/')[-1] for m in metadatas]))

    return answer, sources

In [41]:
if __name__ == "__main__":
    # Ingest all documents
    print("\n" + "="*60)
    print("COOKING ASSISTANT RAG SYSTEM - HUGGING FACE")
    print("="*60 + "\n")

    print("Step 1: Ingesting documents...")
    total_chunks = ingest_all_documents()

    print("\n" + "="*60)
    print("System Ready! Ask cooking questions.")
    print("="*60 + "\n")

    # Example queries
    test_questions = [
        "How do I properly sauté vegetables?",
        "What's the difference between braising and stewing?",
        "What ingredients do I need for chocolate chip cookies?",
        "What are some healthy cooking methods?",
        "Tell me about steaming as a cooking technique",
        "Tell me about cooking technique that I need to know to make friend rice"
    ]

    for question in test_questions:
        print(f"\n{'='*60}")
        print(f"Q: {question}")
        print(f"{'='*60}")

        answer, sources = ask_cooking_assistant(question)

        print(f"\nA: {answer}")
        print(f"\n📚 Sources: {', '.join(sources)}")


COOKING ASSISTANT RAG SYSTEM - HUGGING FACE

Step 1: Ingesting documents...
Loading core cooking technique files...
✓ Loaded 32 chunks from FCS.json
✓ Loaded 15 chunks from healthy_cooking_method.json

Loading recipes from foc folder...
✓ Loaded 33 recipes

Loading CSV files...
✓ Loaded cooking_methods.csv

✓ INGESTION COMPLETE
Total chunks: 239
  - Recipes: 33
  - CSV files: 1
  - Other: 205

System Ready! Ask cooking questions.


Q: How do I properly sauté vegetables?

A: Unnamed: 0: 17
COOKING METHODS: Sautéing
DESCRIPTION: Sautéing is a method in which food is lightly tossed in little oil just enough to cover the base of the pan. The pan is covered with a lid and the flame or intensity of heat is reduced. The product obtained is slightly moist and tender but without any liquid or gravy. Foods cooked by sauteing are generally vegetables. MERITS: Takes less time. Simple technique. Minimum oil is used. DEMERITS: Scorching or burning is possible if not watched closely. Type of method:

In [42]:
question = "what kind of sauce is make with emulsion?"

answer, sources = ask_cooking_assistant(question)

print(f"\nA: {answer}")
print(f"\n📚 Sources: {', '.join(sources)}")


A: Answer: Espagnole sauce, which is a cold emulsion sauce made using carrot, beef stock, chopped onion, tomato puree, garlic, celery, butter, black pepper, flour, bay leaves, and prepared in a pan over low heat for 15 minutes until thickened.

📚 Sources: foc_ingredients_Espagnole_sauce.json, foc_ingredients_Oriental_sauce.json, foc_sections.json
